# EMG Anylsis

You'll learn how to load and analyze EMG signals. 

This demo is a jupyter notebook, i.e. intended to be run step by step.

Author: Eric Einspänner
<br>
Contributor: Nastaran Takmilhomayouni

First version: 6th of July 2023 # to be changed


Copyright 2023 Clinic of Neuroradiology, Magdeburg, Germany

License: Apache-2.0

https://physionet.org/content/emgdb/1.0.0/

https://stackoverflow.com/questions/66727140/how-can-i-read-a-dat-file-together-with-a-hea-file

https://github.com/MIT-LCP/wfdb-python/tree/main/sample-data

## Table Of Contents
0. [Initial Set-Up for Google Colab](#initial-set-up-for-google-colab)
1. [Initial Set-Up (offline)](#initial-set-up-offline)
2. [File Formats](#File-Formats)

## Initial Set-Up for Google Colab
<u> Execute these code blocks just in Google Colab! </u>

In [ ]:
!git clone https://github.com/University-Clinic-of-Neuroradiology/python-bootcamp.git

In [ ]:
import os
import sys
from google.colab import output
output.enable_custom_widget_manager()

sys.path.insert(0,'/content/python-bootcamp/notebooks/TimeAnalysis')
os.chdir(sys.path[0])

In [ ]:
%pip install -q ipympl numpy matplotlib SciPy wfdb biosignalsnotebooks

In [ ]:
import os
import numpy as np
from numpy import linspace, max, min, average, std, sum, sqrt, where, argmax, mean
import matplotlib.pyplot as plt

from scipy.integrate import cumtrapz
from scipy.signal import welch
import wfdb

import biosignalsnotebooks as bsnb                  # biosignalsnotebooks python packag

## Initial Set-Up (offline)

In [ ]:
# Make sure figures appears inline and animations works
# Edit this to ""%matplotlib notebook" when using the "classic" jupyter notebook interface
%matplotlib widget

In [ ]:
import os
import numpy as np
from numpy import linspace, max, min, average, std, sum, sqrt, where, argmax, mean
import matplotlib.pyplot as plt

from scipy.integrate import cumtrapz
from scipy.signal import welch
import wfdb

import biosignalsnotebooks as bsnb                  # biosignalsnotebooks python package

## --- Start notebook ---

# EMG file formats
| Format Name | File Extension | Origin                                         |
|-------------|----------------|------------------------------------------------|
| Text        | .txt           |                                                |
| Header      | .mat/.hea      | Analyze Software, Mayo Clinic                  |

In [ ]:
fs = 1000
dir = os.path.join(os.path.realpath(''),'Data/emg3.txt')
emg_raw = []

with open(dir) as f:
    lines = f.readlines()
for l in lines:
    emg_raw.append(float(l.replace("\n","")))

time = linspace(0, len(emg_raw) / fs, len(emg_raw))

# plot EMG signal
fig = plt.figure()
plt.plot(time, emg_raw)
plt.xlabel('Time (sec)')
plt.ylabel('EMG (mV)')
fig_name = 'rawEMG.png'
fig.set_size_inches(w=11,h=7)
fig.savefig(fig_name)

In [ ]:
emg_detrend = (emg_raw - mean(emg_raw))
burst_begin, burst_end = bsnb.detect_emg_activations(emg_detrend, fs, smooth_level=20, threshold_level=10, 
                                                     time_units=True, plot_result=True)[:2]

In [ ]:
print('EMG Onset Points: ',burst_begin)
print('EMG Offset Points: ', burst_end)
print('Number of Muscular Activations: ', len(burst_begin))

# Bursts Duration
bursts_time = burst_end - burst_begin

# Parameter extraction
max_time = max(bursts_time)
min_time = min(bursts_time)
avg_time = average(bursts_time)
std_time = std(bursts_time)

duration_extract = {'Maximum Activation Time':max_time, 
                     'Minimum Activation Time':min_time, 
                     'Average Activation Time':avg_time, 
                     'Standard deviation Time':std_time}
for x in duration_extract:
    print(x, ': ',"{:.3f}".format(duration_extract[x]),'s')

In [ ]:
# Maximum Value of EMG 
max_sample_value = max(emg_detrend)

# Minimum Value of EMG
min_sample_value = min(emg_detrend)

# Average and Standard Deviation
avg_sample_value = average(emg_detrend)
std_sample_value = std(emg_detrend)

emg_extract = {"Maximum EMG": max_sample_value, 
               "Minimum EMG": min_sample_value, 
               "Average EMG": avg_sample_value, 
               "Standard Deviation EMG": std_sample_value}
for x in emg_extract:
    print(x, ': ',emg_extract[x],'mV')

In [ ]:
# Root Mean Square
rms = sqrt(sum(emg_detrend * emg_detrend) / len(emg_detrend))

# Signal Power Spectrum
f, P = welch(emg_detrend, fs=fs, window='hanning', noverlap=0, nfft=int(256.))

# Total Power and Median Frequency (Frequency that divides the spectrum into two regions with equal power)
area_freq = cumtrapz(P, f, initial=0)
total_power = area_freq[-1]
median_freq = f[where(area_freq >= total_power / 2)[0][0]]
f_max = f[argmax(P)]

print ('Root Mean Square: ', rms,'mV')
print ('Total Power: ', total_power)
print ('Median Frequency: ', median_freq,'Hz')
print ('Maximum Power Frequency: ', f_max, 'Hz')
bsnb.plot_emg_spect_freq(f, P, f_max, median_freq)

In [ ]:
# Demo 1 - Read a WFDB record using the 'rdrecord' function into a wfdb.Record object.
# Plot the signals, and show the data.
record = wfdb.rdrecord('Data/a103l') 
wfdb.plot_wfdb(record=record, title='Record a103l from PhysioNet Challenge 2015') 
display(record.__dict__)

# Can also read the same files hosted on PhysioNet https://physionet.org/content/challenge-2015/1.0.0
# in the /training/ database subdirectory.
record2 = wfdb.rdrecord('a103l', pn_dir='challenge-2015/training/')